In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')#忽略一些警告

In [2]:
train=pd.read_csv("data/train_etl.csv")
train.head()

,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,房屋朝向,居住状态,卧室数量,厅的数量,...,出租方式,区,位置,地铁线路,地铁站点,距离,装修情况,月租金,log_rent,新朝向
0,1,3072,0.128906,2,0.236364,0.008628,东南,0.0,1,1,...,-1.0,11.0,118.0,2.0,40.0,0.764167,-1.0,5.602716,1.887481,东南
1,1,3152,0.132812,1,0.381818,0.017046,东,0.0,1,0,...,-1.0,10.0,100.0,4.0,58.0,0.709167,-1.0,16.977929,2.889145,东
2,1,5575,0.042969,0,0.290909,0.010593,东南,0.0,2,1,...,-1.0,12.0,130.0,5.0,37.0,0.572500,-1.0,8.998302,2.302415,东南
3,1,3103,0.085938,2,0.581818,0.019199,南,0.0,3,2,...,-1.0,7.0,90.0,2.0,63.0,0.658333,-1.0,5.602716,1.887481,南
4,1,5182,0.214844,0,0.545455,0.010427,东北,0.0,2,1,...,-1.0,3.0,31.0,0.0,0.0,2.000000,-1.0,7.300509,2.116317,东北


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196499 entries, 0 to 196498
Data columns (total 21 columns):
时间          196499 non-null int64
小区名         196499 non-null int64
小区房屋出租数量    196499 non-null float64
楼层          196499 non-null int64
总楼层         196499 non-null float64
房屋面积        196499 non-null float64
房屋朝向        196499 non-null object
居住状态        196499 non-null float64
卧室数量        196499 non-null int64
厅的数量        196499 non-null int64
卫的数量        196499 non-null int64
出租方式        196499 non-null float64
区           196499 non-null float64
位置          196499 non-null float64
地铁线路        196499 non-null float64
地铁站点        196499 non-null float64
距离          196499 non-null float64
装修情况        196499 non-null float64
月租金         196499 non-null float64
log_rent    196499 non-null float64
新朝向         196499 non-null object
dtypes: float64(13), int64(6), object(2)
memory usage: 31.5+ MB


## 根据房间,厅,卫,房屋面积构造新特征

In [4]:
train["房+卫+厅"]=train["卧室数量"]+train["厅的数量"]+train["卫的数量"]
train["房/总"]=train["卧室数量"]/(train["房+卫+厅"]+1)
train["卫/总"]=train["卫的数量"]/(train["房+卫+厅"]+1)
train["厅/总"]=train["厅的数量"]/(train["房+卫+厅"]+1)
train['卧室面积']=train['房屋面积']/(train['卧室数量']+1)#加1是为了防止分母=0出现结果为inf无穷大的现象
train['楼层比']=train['楼层']/(train["总楼层"]+1)#加1是为了防止分母=0出现结果为inf无穷大的现象
train['户型']=train[['卧室数量','厅的数量','卫的数量']].apply(lambda x:str(x['卧室数量'])+str(x['厅的数量'])+str(x['卫的数量']),axis=1)

## 构建租金平均值特征

In [5]:
rent_means=train[['小区名','新朝向','地铁站点','位置','log_rent']].groupby(['小区名','新朝向','地铁站点','位置'],as_index=False).mean()

rent_means.columns=['小区名','新朝向','地铁站点','位置','平均值特征1']
train=pd.merge(train,rent_means,how='left',on=['小区名','新朝向','地铁站点','位置'])

In [6]:
rent_means2=train[['小区名','log_rent']].groupby(['小区名'],as_index=False).mean()

rent_means2.columns=['小区名','小区平均值特征']
train=pd.merge(train,rent_means2,how='left',on=['小区名'])

In [7]:
rent_means3=train[['新朝向','log_rent']].groupby(['新朝向'],as_index=False).mean()

rent_means3.columns=['新朝向','朝向平均值特征']
train=pd.merge(train,rent_means3,how='left',on=['新朝向'])

In [8]:
rent_means4=train[['地铁站点','log_rent']].groupby(['地铁站点'],as_index=False).mean()

rent_means4.columns=['地铁站点','站点平均值特征']
train=pd.merge(train,rent_means4,how='left',on=['地铁站点'])

In [9]:
rent_means5=train[['位置','log_rent']].groupby(['位置'],as_index=False).mean()

rent_means5.columns=['位置','位置平均值特征']
train=pd.merge(train,rent_means5,how='left',on=['位置'])

## 构造是否有地铁

In [20]:
train["有地铁"]=(train["地铁站点"]>0).map(int)
train['有地铁'].value_counts()

1    106957
0     89542
Name: 有地铁, dtype: int64

In [21]:
train.columns

Index(['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积', '房屋朝向', '居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', '月租金',
       'log_rent', '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型',
       '平均值特征1', '小区平均值特征', '朝向平均值特征', '站点平均值特征', '位置平均值特征', '有地铁'],
      dtype='object')

## 构造聚类特征

In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
features = train[['房屋面积','卧室数量','厅的数量','卫的数量','距离','楼层比']]
transfer = StandardScaler()
new_feature = transfer.fit_transform(features)

kmeans = KMeans(n_clusters=5)
kmeans.fit(new_feature)
train['聚类特征']=kmeans.predict(new_feature).astype(str)

# 计算每个特征的月租金平均值
cluster_means = train[['聚类特征','log_rent']].groupby('聚类特征',as_index=False).mean()

cluster_means.columns = ['聚类特征','平均值特征2']
cluster_means

,聚类特征,平均值特征2
0,0,1.915702
1,1,2.053872
2,2,1.753141
3,3,2.412708
4,4,2.126248


In [28]:
train=pd.merge(train,cluster_means,how='left',on=['聚类特征'])

## 保存标准化和聚类模型

In [29]:
import pickle
with open('data/kmeans.pkl','wb') as f:
    pickle.dump({
        'std_transfer':transfer,
        'kmeans':kmeans
    },f)

## 构造地铁线路数特征

In [30]:
lines_count1=train[['小区名','地铁线路']].drop_duplicates().groupby('小区名').count()
lines_count2=train[['位置','地铁线路']].drop_duplicates().groupby('位置').count()
lines_count2.columns=['位置线路数']
lines_count1.columns=['小区线路数']

In [31]:
train=pd.merge(train,lines_count1,how='left',on=['小区名'])
train=pd.merge(train,lines_count2,how='left',on=['位置'])

## 去掉出现数量较少的小区

In [33]:
neighbors=train['小区名'].value_counts()
train['新小区名']=train.apply(lambda x: x['小区名'] if neighbors[x['小区名']]>100 else -1,axis=1)
train['小区条数大于100']=train.apply(lambda x: 1 if neighbors[x['小区名']]>100 else 0,axis=1)

In [34]:
train.info()
train['新小区名'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196499 entries, 0 to 196498
Data columns (total 40 columns):
时间           196499 non-null int64
小区名          196499 non-null int64
小区房屋出租数量     196499 non-null float64
楼层           196499 non-null int64
总楼层          196499 non-null float64
房屋面积         196499 non-null float64
房屋朝向         196499 non-null object
居住状态         196499 non-null float64
卧室数量         196499 non-null int64
厅的数量         196499 non-null int64
卫的数量         196499 non-null int64
出租方式         196499 non-null float64
区            196499 non-null float64
位置           196499 non-null float64
地铁线路         196499 non-null float64
地铁站点         196499 non-null float64
距离           196499 non-null float64
装修情况         196499 non-null float64
月租金          196499 non-null float64
log_rent     196499 non-null float64
新朝向          196499 non-null object
房+卫+厅        196499 non-null int64
房/总          196499 non-null float64
卫/总          196499 non-null float64
厅/总          1964

-1       72488
 5512     1880
 1085     1155
 5208     1136
 6221     1066
         ...  
 244       102
 6461      102
 5906      102
 1654      102
 5196      101
Name: 新小区名, Length: 512, dtype: int64

## 转换类型

In [35]:
#将离散特征转换成字符串类型
colunms = ['时间', '小区名', '居住状态', '出租方式', '区','位置','地铁线路','地铁站点','装修情况']
for col in colunms:
    train[col] = train[col].astype(str)

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196499 entries, 0 to 196498
Data columns (total 40 columns):
时间           196499 non-null object
小区名          196499 non-null object
小区房屋出租数量     196499 non-null float64
楼层           196499 non-null int64
总楼层          196499 non-null float64
房屋面积         196499 non-null float64
房屋朝向         196499 non-null object
居住状态         196499 non-null object
卧室数量         196499 non-null int64
厅的数量         196499 non-null int64
卫的数量         196499 non-null int64
出租方式         196499 non-null object
区            196499 non-null object
位置           196499 non-null object
地铁线路         196499 non-null object
地铁站点         196499 non-null object
距离           196499 non-null float64
装修情况         196499 non-null object
月租金          196499 non-null float64
log_rent     196499 non-null float64
新朝向          196499 non-null object
房+卫+厅        196499 non-null int64
房/总          196499 non-null float64
卫/总          196499 non-null float64
厅/总          196499 no

In [38]:
train.describe()

,小区房屋出租数量,楼层,总楼层,房屋面积,卧室数量,厅的数量,卫的数量,距离,月租金,log_rent,...,小区平均值特征,朝向平均值特征,站点平均值特征,位置平均值特征,有地铁,平均值特征2,小区线路数,位置线路数,新小区名,小区条数大于100
count,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,...,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000,196499.000000
mean,0.123924,0.955532,0.408707,0.013105,2.236698,1.299640,1.223843,1.237631,7.948854,2.050408,...,2.050408,2.050408,2.050408,2.050408,0.544313,2.050408,1.016621,2.289564,2024.410547,0.631102
std,0.133264,0.851524,0.183096,0.006074,0.896960,0.613177,0.487261,0.724317,6.310063,0.496835,...,0.393596,0.050089,0.233768,0.303903,0.498034,0.222251,0.127847,0.935287,2248.411915,0.482507
min,0.007812,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001667,0.000000,0.000000,...,0.156815,1.992230,1.221452,1.352958,0.000000,1.753141,1.000000,1.000000,-1.000000,0.000000
25%,0.039062,0.000000,0.290909,0.009268,2.000000,1.000000,1.000000,0.572500,4.923599,1.778944,...,1.791740,1.992230,1.934929,1.836944,0.000000,1.915702,1.000000,2.000000,-1.000000,0.000000
50%,0.082031,1.000000,0.418182,0.012910,2.000000,1.000000,1.000000,0.971667,6.621392,2.030959,...,2.011586,2.027504,1.934929,2.012569,1.000000,2.053872,1.000000,2.000000,1115.000000,1.000000
75%,0.160156,2.000000,0.563636,0.014896,3.000000,2.000000,1.000000,2.000000,8.998302,2.302415,...,2.240552,2.108346,2.124630,2.221863,1.000000,2.126248,1.000000,3.000000,3813.000000,1.000000
max,1.000000,2.000000,1.000000,0.280702,11.000000,8.000000,8.000000,2.000000,100.000000,4.615121,...,4.615121,2.108346,2.858572,4.215430,1.000000,2.412708,2.000000,5.000000,6627.000000,1.000000


In [42]:
#保存处理后的数据
train.to_csv("data/onehot_feature.csv",index=None)

In [41]:
train.columns

Index(['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积', '房屋朝向', '居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', '月租金',
       'log_rent', '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型',
       '平均值特征1', '小区平均值特征', '朝向平均值特征', '站点平均值特征', '位置平均值特征', '有地铁', '聚类特征',
       '平均值特征2', '小区线路数', '位置线路数', '新小区名', '小区条数大于100'],
      dtype='object')

In [ ]:
x_columns=['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型',
       '有地铁']
x=train[x_columns]
y=train['月租金']